In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time
import sys
sys.path.append('.')
import d2p

In [11]:
# transform = transforms.ToTensor() 将图片数据格式转换为tensor
#(H,W,C)(0-255)-->(C,H,W)(0,1)
mninst_train=\
    torchvision.datasets.FashionMNIST(root='./Datasets/FashionMNIST',
                                              train=True,download=True,transform=transforms.ToTensor())
mninst_test=\
    torchvision.datasets.FashionMNIST(root='./Datasets/FashionMNIST',
                                              train=False,download=True,transform=transforms.ToTensor())


Using downloaded and verified file: ./Datasets/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz
Extracting ./Datasets/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./Datasets/FashionMNIST\FashionMNIST\raw


EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
# 本函数已保存在d2pzh包中⽅方便便以后使⽤用
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress',
    'coat','sandal', 'shirt', 'sneaker', 'bag', 'ankleboot']
    return [text_labels[int(i)] for i in labels]
# 本函数已保存在d2pzh包中⽅方便便以后使⽤用
def show_fashion_mnist(images, labels):
    d2p.use_svg_display()
    # 这⾥里里的_表示我们忽略略（不不使⽤用）的变量量
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
    f.imshow(img.view((28, 28)).numpy())
    f.set_title(lbl)
    f.axes.get_xaxis().set_visible(False)
    f.axes.get_yaxis().set_visible(False)
    plt.show()
X, y = [], []
for i in range(10):
    X.append(mnist_train[i][0])
    y.append(mnist_train[i][1])
show_fashion_mnist(X, get_fashion_mnist_labels(y))

In [ ]:
#小批量读取数据
batch_size = 256
if sys.platform.startswith('win'):
    num_workers = 0 # 0表示不不⽤用额外的进程来加速读取数据
else:
    num_workers = 4
train_iter = torch.utils.data.DataLoader(mnist_train,
    batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test,
    batch_size=batch_size, shuffle=False, num_workers=num_workers)
start = time.time()
for X, y in train_iter:
    continue
print('%.2f sec' % (time.time() - start))

In [ ]:
#################softmax从0实现##################
#初始化模型参数
num_inputs = 784
num_outputs = 10
W = torch.tensor(np.random.normal(0, 0.01, (num_inputs,
    num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)

W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)
#保持维度的求和
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(X.sum(dim=0, keepdim=True))
print(X.sum(dim=1, keepdim=True))
#softmax运算
def softmax(X):
    X_exp=X.exp()
    partition=X_exp.sum(dim=1,keepdim=True)
    return X_exp/partition
#定义模型
def net(X):
    return softmax(torch.mm(X.view((-1,num_inputs)),W)+b)
#损失函数
def cross_entropy(y_hat,y):
    return -torch.log(y_hat.gather(1,y.view(-1,1)))
#分类准确率
def accuracy(y_hat,y):
    return (y_hat.argmax(dim=1)==y).float().mean().item()
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n
#训练模型
num_epochs, lr = 5, 0.1
# 本函数已保存在d2pzh包中⽅方便便以后使⽤用
def train_ch3(net, train_iter, test_iter, loss, num_epochs,batch_size,
    params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            if optimizer is None:
                d2p.sgd(params, lr, batch_size)
            else:
                optimizer.step() # “softmax回归的简洁实现”⼀一节将⽤用到
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) ==y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'% (epoch + 1, train_l_sum / n, train_acc_sum / n,test_acc))
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs,batch_size, [W, b], lr)

In [12]:
#yuce
X, y = iter(test_iter).next()
true_labels = d2p.get_fashion_mnist_labels(y.numpy())
pred_labels =d2p.get_fashion_mnist_labels(net(X).argmax(dim=1).numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels,pred_labels)]
d2p.show_fashion_mnist(X[0:9], titles[0:9])

NameError: name 'test_iter' is not defined

In [ ]:
#Softmax的简洁实现
batch_size = 256
train_iter, test_iter = d2p.load_data_fashion_mnist(batch_size)
#定义初始化模型
num_inputs = 784
num_outputs = 10
class LinearNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
    def forward(self, x): # x shape: (batch, 1, 28, 28)
        y = self.linear(x.view(x.shape[0], -1))
        return y
net = LinearNet(num_inputs, num_outputs)
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)
#初始化
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)
#分开定义softmax和交叉熵损失函数会不稳定
loss = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(net.parameters(), lr=0.1) 
num_epochs = 5
d2p.train_ch3(net, train_iter, test_iter, loss, num_epochs,
batch_size, None, None, optimizer)